# RC Circuit Response Prediction using LSTM
This notebook demonstrates how to:
- Generate synthetic time-series data for an RC circuit.
- Preprocess the data.
- Build and train an LSTM neural network.
- Evaluate and visualize predictions vs actual responses.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

In [ ]:
# Step 1: Generate Synthetic Time-Series Data for RC Circuit
np.random.seed(42)
num_samples = 1000
num_timesteps = 50  # predict 50 time points

# Random R, C, V_in values
R_values = np.random.uniform(100, 1000, num_samples)
C_values = np.random.uniform(1e-6, 1e-3, num_samples)
V_in_values = np.random.uniform(5, 15, num_samples)

# Time vector (0 to 0.05s)
time = np.linspace(0, 0.05, num_timesteps)

# Generate responses
responses = []
for i in range(num_samples):
    R, C, Vin = R_values[i], C_values[i], V_in_values[i]
    V_out = Vin * (1 - np.exp(-time / (R * C)))
    responses.append(V_out)
responses = np.array(responses)

# Features: R, C, Vin repeated for each timestep
X = np.column_stack((R_values, C_values, V_in_values))
X_norm = (X - X.mean(axis=0)) / X.std(axis=0)

# Repeat normalized features across timesteps for LSTM input
X_seq = np.repeat(X_norm[:, np.newaxis, :], num_timesteps, axis=1)

# Target: responses
Y_seq = responses

In [ ]:
# Step 2: Build LSTM Model
model = Sequential([
    LSTM(32, input_shape=(num_timesteps, 3), return_sequences=True),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')

In [ ]:
# Step 3: Train Model
history = model.fit(X_seq, Y_seq[..., np.newaxis], epochs=50, batch_size=32, verbose=1)

In [ ]:
# Step 4: Test Prediction
# Example: R=500Ω, C=500uF, V_in=10V
test_R, test_C, test_Vin = 500, 500e-6, 10
X_test = np.array([[test_R, test_C, test_Vin]])
X_test_norm = (X_test - X.mean(axis=0)) / X.std(axis=0)
X_test_seq = np.repeat(X_test_norm[:, np.newaxis, :], num_timesteps, axis=1)

predicted_response = model.predict(X_test_seq)[0,:,0]
actual_response = test_Vin * (1 - np.exp(-time / (test_R * test_C)))

# Plot Prediction vs Actual
plt.figure(figsize=(8,5))
plt.plot(time, actual_response, label='Actual Response')
plt.plot(time, predicted_response, label='Predicted Response', linestyle='--')
plt.xlabel('Time (s)')
plt.ylabel('Voltage (V)')
plt.title('RC Circuit Response: Actual vs Predicted')
plt.legend()
plt.show()

print(f"Actual final voltage: {actual_response[-1]:.4f} V")
print(f"Predicted final voltage: {predicted_response[-1]:.4f} V")